# Lung cancer classification with Graph Convolutional Networks

In [6]:
import os
if not os.path.exists("README.md"):
    os.chdir("../")

import pandas as pd
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

# for the ML part
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

# for the graph part
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx, from_networkx

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from collections import Counter

from scripts.gcn import GCN, train, test, train_loop
BATCH_SIZE = 16


## Data preprocessing

### import dataset from csv files


In [7]:
degenes = pd.read_csv('./data/final/degenes.csv', index_col=0)
pdata = pd.read_csv('./data/final/pdata_nan_filled.csv', index_col=0)

degenes_t = degenes.T
degenes_t.columns = [x.split('///')[0] for x in degenes_t.columns]
degenes = degenes_t.T
degenes = degenes/10
degenes_t = degenes.T/10

matrix = pd.read_csv('data/final/adj_matrix.csv', index_col=0)
matrix = matrix.drop('cancer_status', axis=1).drop('cancer_status', axis=0)

degenes_t.head(3)

,GSDMB,TXN,SLC5A1,NSUN3,HSPA13,SOX9,ZKSCAN5,AMACR,LOC101060275,LOC101928625,...,CLGN,TMEM242,SERPINI1,TTC3,CD74,ZBTB18,REC8,MARC2,YWHAE,PRSS12
GSM93997,0.140096,0.235407,0.171271,0.183405,0.155741,0.132806,0.133945,0.136854,0.201459,0.188271,...,0.158699,0.170411,0.169404,0.224789,0.244948,0.113263,0.215222,0.155076,0.162765,0.162176
GSM94019,0.139721,0.220102,0.178908,0.183668,0.138633,0.150779,0.125344,0.139069,0.217829,0.154159,...,0.142984,0.169939,0.157350,0.218183,0.242422,0.110154,0.208866,0.156494,0.135190,0.152349
GSM94020,0.143053,0.202522,0.179960,0.181274,0.130426,0.155367,0.127912,0.137341,0.207420,0.142284,...,0.128292,0.176204,0.156956,0.206328,0.238198,0.114715,0.211555,0.152307,0.118662,0.161834


In [8]:
#drop nan in pdata
pdata = pdata.dropna(axis=0)
pdata

,age,biomarker_score,cancer_status,gender,>3cm,packyears,hemopytsis,lymphadenopathy,smoking_status,subjective_assessment
GSM93997,34.0,-2.253540,0.0,1.0,0.0,17.0,0.0,0.0,0.0,0.0
GSM94019,63.0,8.900589,1.0,1.0,1.0,75.0,0.0,1.0,0.0,1.0
GSM94020,61.0,9.954619,1.0,1.0,1.0,80.0,0.0,1.0,0.0,1.0
GSM94021,69.0,-3.460146,1.0,1.0,0.0,70.0,0.0,1.0,1.0,1.0
GSM94022,61.0,1.543728,1.0,1.0,0.0,80.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
GSM98871,62.0,6.628599,1.0,1.0,0.0,67.0,0.0,1.0,0.0,1.0
GSM98872,62.0,3.834675,1.0,1.0,1.0,72.0,0.0,1.0,0.0,1.0
GSM98873,66.0,-0.549258,1.0,1.0,1.0,61.0,0.0,1.0,0.0,1.0
GSM98874,65.0,6.642173,1.0,1.0,1.0,65.0,0.0,1.0,0.0,1.0


### MinMaxScaler and StandardScaler pass

In [9]:
# min max scale
degenes_scaled = pd.DataFrame(degenes, index=degenes.index, columns=degenes.columns)
#degenes_scaled = degenes_scaled.applymap(lambda x: np.exp(x))

scaler = StandardScaler()
degenes_scaled = pd.DataFrame(scaler.fit_transform(degenes_scaled), index=degenes_scaled.index, columns=degenes_scaled.columns)

mmscaler = MinMaxScaler()
degenes_scaled = pd.DataFrame(mmscaler.fit_transform(degenes_scaled), index=degenes_scaled.index, columns=degenes_scaled.columns)

# minmax pdata
pdata_scaled = pd.DataFrame(pdata, index=pdata.index, columns=pdata.columns)
pdata_scaled = pd.DataFrame(scaler.fit_transform(pdata_scaled), index=pdata_scaled.index, columns=pdata_scaled.columns)
pdata_scaled = pd.DataFrame(mmscaler.fit_transform(pdata_scaled), index=pdata_scaled.index, columns=pdata_scaled.columns)

degenes_scaled.head(3)

,GSM93997,GSM94019,GSM94020,GSM94021,GSM94022,GSM94023,GSM94024,GSM94025,GSM94026,GSM94027,...,GSM98797,GSM98798,GSM98799,GSM98800,GSM98801,GSM98871,GSM98872,GSM98873,GSM98874,GSM98875
GSDMB,0.206948,0.209339,0.233461,0.320483,0.206750,0.211192,0.189032,0.115259,0.224393,0.178037,...,0.203517,0.188561,0.255286,0.272566,0.155407,0.216968,0.293979,0.241238,0.261283,0.237032
TXN,0.845322,0.751053,0.657096,0.793263,0.816640,0.737990,0.832710,0.820180,0.822031,0.839329,...,0.783206,0.737515,0.788049,0.819971,0.799605,0.566921,0.341013,0.715559,0.461597,0.785306
SLC5A1,0.415754,0.473435,0.496373,0.494284,0.472976,0.437425,0.493254,0.478358,0.453937,0.490949,...,0.446771,0.438553,0.476721,0.452521,0.411302,0.421924,0.508630,0.443029,0.481669,0.443907


In [10]:
pdata_scaled.head(3)

,age,biomarker_score,cancer_status,gender,>3cm,packyears,hemopytsis,lymphadenopathy,smoking_status,subjective_assessment
GSM93997,0.174603,0.409506,0.0,1.0,0.0,0.094444,0.0,0.0,0.0,0.0
GSM94019,0.634921,0.721167,1.0,1.0,1.0,0.416667,0.0,1.0,0.0,1.0
GSM94020,0.603175,0.750618,1.0,1.0,1.0,0.444444,0.0,1.0,0.0,1.0


## Building graph structure from adjacency matrix

In [12]:
graphs = {}

for i in range(0, len(degenes_scaled.columns)):
    G = nx.from_pandas_adjacency(matrix)
    G.remove_nodes_from(list(nx.isolates(G)))
    nx.set_node_attributes(G, degenes_scaled.iloc[:,i].to_dict(), 'x')

    for edge in G.edges:
        G.edges[edge]['weight'] = 1

    graphs[degenes_scaled.columns[i]] = G

### Create pytorch graph structure

In [13]:
# create a geometric data object from the networkx for each graph
data_list = []
for key, value in graphs.items():
    try:
        cs = pdata.loc[key, 'cancer_status']

        d = from_networkx(value)
        d.x = torch.tensor([d[1]['x'] for d in value.nodes(data=True)], dtype=torch.float32)
        d.x = d.x.view(-1, 1)

        target = torch.tensor([[0, 1]], dtype=torch.float32) if cs == 1 else torch.tensor([[1, 0]], dtype=torch.float32)
        additional_features = pdata.loc[key].drop(['cancer_status','subjective_assessment'], axis=0)
        additional_features = additional_features.to_frame().T
        additional_features = additional_features.astype('float32')

        d.y = [target, torch.tensor(additional_features.values)]

        data_list.append(d)
    except:
        KeyError

### Split into train and test

In [21]:
# split in train validation and test
train_data, test_data = train_test_split(data_list, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

len(train_data), len(test_data), len(val_data)

(134, 29, 29)

### Create torch DataLoaders

In [22]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [23]:
from collections import Counter
train_cancer_status = [torch.argmax(d.y[0]).item() for d in train_data]
val_cancer_status = [torch.argmax(d.y[0]).item() for d in val_data]
test_cancer_status = [torch.argmax(d.y[0]).item() for d in test_data]

print('train_data: ', Counter(train_cancer_status))
print('val_data: ', Counter(val_cancer_status))
print('test_data: ', Counter(test_cancer_status))

train_data:  Counter({1: 73, 0: 61})
val_data:  Counter({1: 16, 0: 13})
test_data:  Counter({1: 18, 0: 11})


## Now, graph classification

In [50]:
# autoreload 
%load_ext autoreload
%autoreload 2
from scripts.gcn import GCN, EarlyStopping, train_loop, test, GCNClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
model = GCN(hidden_channels=256)
print(model)

# define training loop
device = torch.device("cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)
criterion = torch.nn.MSELoss()

GCN(
  (conv1): GCNConv(1, 256)
  (lin1): Linear(in_features=264, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=2, bias=True)
)


In [52]:
train_loop(
    model, 
    criterion, 
    optimizer, 
    train_loader, 
    val_loader, 
    epochs=1000,
    early_stopping=EarlyStopping(patience=50, delta=0.0001, verbose=True),
    verbose=True
)

Epoch: 000, Train Loss: 0.2020, Train Acc: 0.7463, Val Loss: 0.1686, Val Acc: 0.8276
Epoch: 001, Train Loss: 0.1847, Train Acc: 0.7612, Val Loss: 0.1363, Val Acc: 0.8621
Epoch: 002, Train Loss: 0.1736, Train Acc: 0.7463, Val Loss: 0.1166, Val Acc: 0.8621
Epoch: 003, Train Loss: 0.1688, Train Acc: 0.7761, Val Loss: 0.1135, Val Acc: 0.8621
Epoch: 004, Train Loss: 0.1651, Train Acc: 0.7687, Val Loss: 0.1087, Val Acc: 0.8966
Epoch: 005, Train Loss: 0.1627, Train Acc: 0.7761, Val Loss: 0.1088, Val Acc: 0.8966
Epoch: 006, Train Loss: 0.1597, Train Acc: 0.7761, Val Loss: 0.1078, Val Acc: 0.8621
Epoch: 007, Train Loss: 0.1570, Train Acc: 0.7761, Val Loss: 0.1062, Val Acc: 0.8621
Epoch: 008, Train Loss: 0.1552, Train Acc: 0.7836, Val Loss: 0.1047, Val Acc: 0.8966
Epoch: 009, Train Loss: 0.1523, Train Acc: 0.7836, Val Loss: 0.1073, Val Acc: 0.8966
Epoch: 010, Train Loss: 0.1495, Train Acc: 0.7836, Val Loss: 0.1042, Val Acc: 0.8966
Epoch: 011, Train Loss: 0.1462, Train Acc: 0.7836, Val Loss: 0.10

128 hidden units, 0.2 dropout, 1e-3 L2, 0.001 learning rate
Best train loss: 0.1018	Best val loss: 0.1436

In [53]:
acc, loss = test(test_loader, model, criterion)
print(f'Test Accuracy: {acc:.4f}')

Test Accuracy: 0.7586


In [35]:
dev_set = train_data.copy()
for x in val_data:
    dev_set.append(x)

dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=False)

In [54]:
model = GCN(hidden_channels=64)
print(model)
torch.manual_seed(1)

# define training loop
device = torch.device("cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)
criterion = torch.nn.MSELoss()

train_loop(
    model,
    criterion,
    optimizer,
    dev_loader,
    dev_loader,
    epochs=1000,
    verbose=True,
    min_loss = 0.08
)

GCN(
  (conv1): GCNConv(1, 64)
  (lin1): Linear(in_features=72, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=2, bias=True)
)
Epoch: 000, Train Loss: 0.1837, Train Acc: 0.7730, Val Loss: 0.1837, Val Acc: 0.7730
Epoch: 001, Train Loss: 0.1755, Train Acc: 0.7607, Val Loss: 0.1755, Val Acc: 0.7607
Epoch: 002, Train Loss: 0.1662, Train Acc: 0.7853, Val Loss: 0.1662, Val Acc: 0.7853
Epoch: 003, Train Loss: 0.1577, Train Acc: 0.7853, Val Loss: 0.1577, Val Acc: 0.7853
Epoch: 004, Train Loss: 0.1525, Train Acc: 0.7914, Val Loss: 0.1525, Val Acc: 0.7914
Epoch: 005, Train Loss: 0.1466, Train Acc: 0.8037, Val Loss: 0.1466, Val Acc: 0.8037
Epoch: 006, Train Loss: 0.1422, Train Acc: 0.7975, Val Loss: 0.1422, Val Acc: 0.7975
Epoch: 007, Train Loss: 0.1418, Train Acc: 0.8037, Val Loss: 0.1418, Val Acc: 0.8037
Epoch: 008, Train Loss: 0.1363, Train Acc: 0.8037, Val Loss: 0.1363, Val Acc: 0.8037
Epoch: 009, Train Loss: 0.1321, Train Acc: 0.8098, Val Loss: 0.1321, Val Acc: 0.8

In [55]:
# test on test set
acc, loss = test(test_loader, model, criterion)
print(f'Test Accuracy: {acc:.4f}')

Test Accuracy: 0.8621


### K-Fold Cross Validation

In [81]:
from sklearn.model_selection import KFold

# autoreload 
%load_ext autoreload
%autoreload 2
from scripts.gcn import GCN, EarlyStopping, train_loop, test, GCNClassifier

# lucky config K=3, rs=43, 256 hidden, 0.01 lr, 1e-3 wd
kf = KFold(n_splits=3, shuffle=True, random_state=43)
accuracies =[]
losses = []
counter = 1
torch.manual_seed(42)
for train_index, test_index in kf.split(data_list):

    train_data = [data_list[i] for i in train_index]
    # append the noisy data to the train data

    train_data_noisy = train_data.copy()
    
    """
    for t in train_data_noisy:
        t.x = t.x + (0.5**0.5)*torch.randn(t.x.shape)

    for t in train_data_noisy:
        train_data.append(t)
    """

    test_data = [data_list[i] for i in test_index]

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

    model = GCN(hidden_channels=256)
    print(model)

    # define training loop
    device = torch.device("cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)
    criterion = torch.nn.MSELoss()

    train_loop(
        model,
        criterion,
        optimizer,
        train_loader,
        test_loader,
        epochs=1000,
        verbose=False,
        min_loss=0.1
    )
    model.eval()
    acc, loss = test(test_loader, model, criterion)
    accuracies.append(acc)
    losses.append(loss)
    print(f'> Fold {counter} trained. Test accuracy: {acc:.3f}\tTest loss {loss:.3f}')
    counter += 1

# print mean accuracy and loss
print('-'*20)
print('REPORT')
print(f'Mean accuracy: {np.mean(accuracies):.3f} ')
print(f'Mean loss: {np.mean(losses):.3f} ')
print('-'*20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GCN(
  (conv1): GCNConv(1, 256)
  (lin1): Linear(in_features=264, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=2, bias=True)
  (bn1): BatchNorm(256)
)
Min Loss reached: Epoch: 100, Train Loss: 0.0994, Train Acc: 0.8516
> Fold 1 trained. Test accuracy: 0.922	Test loss 0.081
GCN(
  (conv1): GCNConv(1, 256)
  (lin1): Linear(in_features=264, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=2, bias=True)
  (bn1): BatchNorm(256)
)
Min Loss reached: Epoch: 026, Train Loss: 0.0990, Train Acc: 0.8828
> Fold 2 trained. Test accuracy: 0.812	Test loss 0.137
GCN(
  (conv1): GCNConv(1, 256)
  (lin1): Linear(in_features=264, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=2, bias=True)
  (bn1): BatchNorm(256)
)
Min Loss reached: Epoch: 023, Train Loss: 0.0992, Train Acc: 0.8984
> Fold 3 trained. Test accuracy: 0.844	Test loss 0.135
---

human protein atlas